In [1]:
from ucimlrepo import fetch_ucirepo 
from sklearn.ensemble import VotingRegressor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression,ElasticNet
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score,accuracy_score,log_loss,r2_score
from sklearn.model_selection import train_test_split,StratifiedKFold , GridSearchCV
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import make_column_transformer ,make_column_selector
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

In [3]:
insurance = pd.read_csv('insurance.csv')

In [4]:
X = insurance.drop('charges',axis=1)
y = insurance['charges']

In [5]:
X_train , X_test , y_train , y_test = train_test_split(X,y,random_state=24,test_size=0.3)

In [10]:
ohe = OneHotEncoder(handle_unknown = 'ignore')
ct = make_column_transformer((ohe,make_column_selector(dtype_include=object)) ,
                                   ('passthrough',make_column_selector(dtype_exclude=object)),
                                   verbose_feature_names_out=False)
dtr = DecisionTreeRegressor(random_state=24)
pipe_dtr = Pipeline([('CT',ct),('DT',dtr)])

In [11]:
lr = LinearRegression()
pipe_lr = Pipeline([('CT',ct),('LR',lr)])

In [12]:
el = ElasticNet(random_state=24)
pipe_el = Pipeline([('CT',ct),('EL',el)])

In [14]:
pipe_dtr.fit(X_train,y_train)
y_pred_dtr = pipe_dtr.predict(X_test)
print(r2_score(y_test,y_pred_dtr))

0.7035862006072617


In [15]:
pipe_lr.fit(X_train,y_train)
y_pred_lr = pipe_lr.predict(X_test)
print(r2_score(y_test,y_pred_lr))

0.7665391799816873


In [16]:
pipe_el.fit(X_train,y_train)
y_pred_el = pipe_el.predict(X_test)
print(r2_score(y_test,y_pred_el))

0.4967365055710605


In [19]:
voting = VotingRegressor([('LR',pipe_lr),('EL',pipe_el),('DTR',pipe_dtr)])
voting.fit(X_train,y_train)

VotingRegressor(estimators=[('LR',
                             Pipeline(steps=[('CT',
                                              ColumnTransformer(transformers=[('onehotencoder',
                                                                               OneHotEncoder(handle_unknown='ignore'),
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x0000020EC63FD970>),
                                                                              ('passthrough',
                                                                               'passthrough',
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x0000020EC63FD730>)],
                                                                verb...
                                              ColumnTransformer(transformers=[('onehotencoder',
                                                                               OneHotEncoder(handle_unknown='ignore'),
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x0000020EC63FD970>),
                                                                              ('passthrough',
                                                                               'passthrough',
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x0000020EC63FD730>)],
                                                                verbose_feature_names_out=False)),
                                             ('DT',
                                              DecisionTreeRegressor(random_state=24))]))])

In [20]:
y_pred_voting = voting.predict(X_test)

In [21]:
print(r2_score(y_test,y_pred_voting))

0.7686797982256638


In [26]:
voting = VotingRegressor([('LR',pipe_lr),('EL',pipe_el),('DTR',pipe_dtr)],
                        weights=[0.4,0.2,0.4])  # weights are normalized on its own

In [27]:
voting.fit(X_train,y_train)

VotingRegressor(estimators=[('LR',
                             Pipeline(steps=[('CT',
                                              ColumnTransformer(transformers=[('onehotencoder',
                                                                               OneHotEncoder(handle_unknown='ignore'),
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x0000020EC63FD970>),
                                                                              ('passthrough',
                                                                               'passthrough',
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x0000020EC63FD730>)],
                                                                verb...
                                              ColumnTransformer(transformers=[('onehotencoder',
                                                                               OneHotEncoder(handle_unknown='ignore'),
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x0000020EC63FD970>),
                                                                              ('passthrough',
                                                                               'passthrough',
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x0000020EC63FD730>)],
                                                                verbose_feature_names_out=False)),
                                             ('DT',
                                              DecisionTreeRegressor(random_state=24))]))],
                weights=[0.4, 0.2, 0.4])

In [28]:
y_pred = voting.predict(X_test)

In [29]:
print(r2_score(y_test,y_pred))

0.7911541968084609
